In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np




In [ ]:
username = 'postgres'
password = 'XXXXXXXXX'
database = 'airlines'
host = 'localhost'
port = 5432  

connection_string = f"postgresql://{username}:{password}@{host}:{port}/{database}"
engine = create_engine(connection_string)

flight_df_raw2 = pd.read_sql_table('flight', con=engine)

print(flight_df_raw2.head())

In [ ]:
flight_df_cleaned = flight_df_raw2[
    ~((flight_df_raw2['YEAR'] == 2020) | (flight_df_raw2['CANCELLED'] == 1))
]

print(f"Liczba wierszy przed oczyszczeniem: {len(flight_df_raw2)}")
print(f"Liczba wierszy po oczyszczeniu: {len(flight_df_cleaned)}")



In [ ]:
flight_df_cleaned.rename(columns={'DEP_DELAY_NEW': 'DEP_DELAY'}, inplace=True)

print(flight_df_cleaned.columns)

In [ ]:
dep_delay_statistics_df = flight_df_cleaned['DEP_DELAY'].describe().round(2)

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(flight_df_cleaned['DEP_DELAY'], bins=np.arange(0, flight_df_cleaned['DEP_DELAY'].max() + 10, 10), edgecolor='black')
plt.title('Histogram dla całej kolumny DEP_DELAY')
plt.xlabel('Opóźnienie (minuty)')
plt.ylabel('Częstotliwość')
plt.grid(True)
plt.show()

In [ ]:
filtered_dep_delay = flight_df_cleaned[flight_df_cleaned['DEP_DELAY'] > 0]

plt.figure(figsize=(10, 6))
plt.hist(filtered_dep_delay['DEP_DELAY'], bins=np.arange(0, filtered_dep_delay['DEP_DELAY'].max() + 10, 10), edgecolor='black')
plt.title('Histogram dla dep_delay > 0')
plt.xlabel('Opóźnienie (minuty)')
plt.ylabel('Częstotliwość')
plt.grid(True)
plt.show()


In [ ]:
percentile_95 = np.percentile(flight_df_cleaned['DEP_DELAY'], 95)

filtered_dep_delay = flight_df_cleaned[(flight_df_cleaned['DEP_DELAY'] > 0) & 
                                       (flight_df_cleaned['DEP_DELAY'] < percentile_95)]

plt.figure(figsize=(10, 6))
plt.hist(filtered_dep_delay['DEP_DELAY'], bins=np.arange(0, percentile_95 + 10, 10), edgecolor='black')
plt.title(f'Histogram dla dep_delay > 0 i dep_delay < {percentile_95} (95. percentyl)')
plt.xlabel('Opóźnienie (minuty)')
plt.ylabel('Częstotliwość')
plt.grid(True)
plt.show()


In [ ]:
flight_df_cleaned['is_delayed'] = flight_df_cleaned['DEP_DELAY'] > 15

delayed_ratio = flight_df_cleaned['is_delayed'].mean() * 100

delayed_ratio = round(delayed_ratio, 2)

print(f"Procent opóźnionych lotów: {delayed_ratio}%")


In [ ]:
flight_df_cleaned['is_delayed'] = flight_df_cleaned['DEP_DELAY'] > 15

flight_delays_by_month_df = flight_df_cleaned.groupby('MONTH')['is_delayed'].mean() * 100

plt.figure(figsize=(10, 6))
flight_delays_by_month_df.plot(kind='bar', color='skyblue', edgecolor='black')

plt.title('Odsetek opóźnionych lotów w zależności od miesiąca kalendarzowego', fontsize=14)
plt.xlabel('Miesiąc', fontsize=12)
plt.ylabel('Procent opóźnionych lotów', fontsize=12)
plt.xticks(ticks=range(12), labels=['Styczeń', 'Luty', 'Marzec', 'Kwiecień', 'Maj', 'Czerwiec', 'Lipiec', 'Sierpień', 'Wrzesień', 'Październik', 'Listopad', 'Grudzień'], rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()


In [ ]:
flight_df_cleaned['is_delayed'] = flight_df_cleaned['DEP_DELAY'] > 15

flight_delays_by_weekday_df = flight_df_cleaned.groupby('DAY_OF_WEEK')['is_delayed'].mean() * 100

plt.figure(figsize=(10, 6))
flight_delays_by_weekday_df.plot(kind='bar', color='skyblue', edgecolor='black')

plt.title('Odsetek opóźnionych lotów w zależności od dnia tygodnia', fontsize=14)
plt.xlabel('Dzień tygodnia', fontsize=12)
plt.ylabel('Procent opóźnionych lotów', fontsize=12)
plt.xticks(ticks=range(7), labels=['Poniedziałek', 'Wtorek', 'Środa', 'Czwartek', 'Piątek', 'Sobota', 'Niedziela'], rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()


In [ ]:
flight_df_cleaned['is_weekend'] = flight_df_cleaned['DAY_OF_WEEK'].isin([6, 7])

flight_delays_by_weekend_df = flight_df_cleaned.groupby('is_weekend')['is_delayed'].mean() * 100

flight_delays_by_weekend_df = flight_delays_by_weekend_df.round(2)

plt.figure(figsize=(8, 5))
flight_delays_by_weekend_df.plot(kind='bar', color=['skyblue', 'lightcoral'], edgecolor='black')

plt.title('Odsetek opóźnionych lotów w weekend vs. dni robocze', fontsize=14)
plt.xlabel('Czy to weekend?', fontsize=12)
plt.ylabel('Procent opóźnionych lotów', fontsize=12)
plt.xticks(ticks=[0, 1], labels=['Dni robocze', 'Weekend'], rotation=0)
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()


In [ ]:
flight_distance_analysis_df = flight_df_cleaned['DISTANCE'].describe(percentiles=[.1, .25, .5, .75, .9, .95, .99]).round(2)

print(flight_distance_analysis_df)

sample_df = flight_df_cleaned.sample(n=10000, random_state=42)
plt.figure(figsize=(10, 6))
plt.scatter(sample_df['DISTANCE'], sample_df['DEP_DELAY'], alpha=0.5, color='blue', s=1)
plt.title('Zależność między odległością lotu a opóźnieniem', fontsize=14)
plt.xlabel('Odległość (miles)', fontsize=12)
plt.ylabel('Opóźnienie (minuty)', fontsize=12)
plt.grid(True)
plt.show()

distance_95_percentile = np.percentile(flight_df_cleaned['DISTANCE'], 95)
flight_df_corrected = flight_df_cleaned[flight_df_cleaned['DISTANCE'] <= distance_95_percentile]


flight_df_corrected['distance_agg'] = (flight_df_corrected['DISTANCE'] // 100) * 100


flight_delays_by_distance_agg_df = flight_df_corrected.groupby('distance_agg')['is_delayed'].mean() * 100
flight_delays_by_distance_agg_df = flight_delays_by_distance_agg_df.round(2)

plt.figure(figsize=(12, 6))
flight_delays_by_distance_agg_df.plot(kind='bar', color='lightblue', edgecolor='black')

plt.title('Odsetek opóźnionych lotów w zależności od koszyka odległości', fontsize=14)
plt.xlabel('Koszyk odległości (miles)', fontsize=12)
plt.ylabel('Procent opóźnionych lotów', fontsize=12)
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()


In [ ]:
flight_distance_by_distance_group = flight_df_cleaned.groupby('DISTANCE_GROUP')['DISTANCE'].agg(['min', 'max']).reset_index()

print(flight_distance_by_distance_group)

flight_df_cleaned['is_delayed'] = flight_df_cleaned['DEP_DELAY'] > 0  

flight_delays_by_distance_group_df = flight_df_cleaned.groupby('DISTANCE_GROUP')['is_delayed'].mean().reset_index()
flight_delays_by_distance_group_df['is_delayed'] = flight_delays_by_distance_group_df['is_delayed'].round(2) * 100  

print(flight_delays_by_distance_group_df)

plt.figure(figsize=(12, 6))
plt.bar(flight_delays_by_distance_group_df['DISTANCE_GROUP'], flight_delays_by_distance_group_df['is_delayed'], color='lightblue', edgecolor='black')

plt.title('Prawdopodobieństwo opóźnienia w zależności od grupy odległościowej', fontsize=14)
plt.xlabel('Grupa odległościowa', fontsize=12)
plt.ylabel('Prawdopodobieństwo opóźnienia (%)', fontsize=12)
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()
